In [1]:
using Revise
include("EFITModule/EFIT.jl")
include("EFITModule/ARBSCATT.jl")

using ParallelStencil
using ParallelStencil.FiniteDifferences3D
using GLMakie
ParallelStencil.@reset_parallel_stencil()

USE_GPU = false
@static if USE_GPU
    @init_parallel_stencil(CUDA, Float32, 3)
else
    @init_parallel_stencil(Threads, Float32, 3)
end

In [ ]:
matGrid, params = Main.ARBSCATT.loadARBSCATT("TestFiles");
transducer = Main.ARBSCATT.loadTransducers("TestFiles")

In [ ]:
params

In [ ]:
mat = Main.EFIT.IsoMat(params["λ"],params["μ"],params["ρ"],params["λ"] + 2*params["μ"])

In [ ]:
materials = [Main.EFIT.IsoMats["LightweightGeneric"],mat]



dx = params["ds"]
dt = params["dt"]


grid = Main.EFIT.EFITGrid(matGrid.+1,materials,dt,dx);

In [ ]:
path = "/media/data/2022 Data/RailTest_2,5MHz_14_May"
touch("$path/test.txt")

In [ ]:
fig,ax,plt = volume(Array(grid.vx[1:2:end,1:2:end,1:2:end]),algorithm=:mip,colorrange = (0, 0.01),colormap=:curl, transparency=true)

In [ ]:
sx = 866
sy = 358
sz = 715
r = 45

rx = sx:sx
ry = sy-r:sy+r
rz = sz-r:sz+r

nSteps = 3000
for i = 1:nSteps
    if i % 10 == 0
        println("Saving at $i of $nSteps")
        Main.EFIT.writeToBOV(i*dt,i,grid,directory=path)
        plt.volume = sqrt.(grid.vx[1:2:end,1:2:end,1:2:end].^2 .+ grid.vy[1:2:end,1:2:end,1:2:end].^2 .+ grid.vz[1:2:end,1:2:end,1:2:end].^2)
    end
    println("$i of $nSteps")
    Main.EFIT.SimStep!(grid)
    if i < transducer[1].drivelen
        drive = transducer[1].drivef[i]*grid.dtds /params["ρ"]
    else
        drive = 0

    end
    @parallel (rx,ry,rz) Main.EFIT.applySource!(grid.vx,grid.vy,grid.vz, drive, Data.Number(0.0), Data.Number(0.0))
end